In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

### Load the player data

In [2]:
phs_2021_1 = pd.read_csv("data/phs_2021_1.csv")
phs_2020_1 = pd.read_csv("data/phs_2020_1.csv")
phs_2020_2 = pd.read_csv("data/phs_2020_2.csv")
phs_df = pd.concat([phs_2021_1, phs_2020_1, phs_2020_2])
phs_df.head()

,start_time,esports_match_id,tournament_title,map_type,map_name,player_name,team_name,stat_name,hero_name,stat_amount
0,2021-04-16 19:08:52,37234,OWL 2021,CONTROL,Busan,Doha,Dallas Fuel,All Damage Done,All Heroes,13900.680095
1,2021-04-16 19:08:52,37234,OWL 2021,CONTROL,Busan,Doha,Dallas Fuel,Assists,All Heroes,8.000000
2,2021-04-16 19:08:52,37234,OWL 2021,CONTROL,Busan,Doha,Dallas Fuel,Average Time Alive,All Heroes,56.481102
3,2021-04-16 19:08:52,37234,OWL 2021,CONTROL,Busan,Doha,Dallas Fuel,Barrier Damage Done,All Heroes,1495.492155
4,2021-04-16 19:08:52,37234,OWL 2021,CONTROL,Busan,Doha,Dallas Fuel,Damage - Quick Melee,All Heroes,60.000000


### Data Cleaning

In [3]:
phs_drop_allheros = phs_df[phs_df["hero_name"]!="All Heroes"]
heroset_df = phs_drop_allheros.groupby(["esports_match_id", "map_name", "team_name"])["hero_name"].agg([lambda x: frozenset(x)]).reset_index().rename(columns={"<lambda>": "hero_used"})
heroset_df.head()

,esports_match_id,map_name,team_name,hero_used
0,30991,Eichenwalde,Paris Eternal,"(Doomfist, Moira, Baptiste, Mei, Wrecking Ball..."
1,30991,Eichenwalde,Toronto Defiant,"(Wrecking Ball, Mei, Lúcio, McCree, Reaper, D...."
2,30991,Havana,Paris Eternal,"(Doomfist, Moira, Wrecking Ball, Mei, Lúcio, M..."
3,30991,Havana,Toronto Defiant,"(Moira, Mei, Lúcio, Reaper, D.Va, Ana, Reinhar..."
4,30991,Horizon Lunar Colony,Paris Eternal,"(Doomfist, Moira, Baptiste, Mei, Wrecking Ball..."


In [4]:
# aggregate heroes used for each team in each map
map_team_heroes_dataset = heroset_df.groupby(["map_name", "team_name"])["hero_used"].agg([lambda x: list(x)]).reset_index().rename(columns={"<lambda>": "hero_used"})
# remove two solo maps
map_team_heroes_dataset = map_team_heroes_dataset[
    (map_team_heroes_dataset["map_name"]!="Castillo") &
    (map_team_heroes_dataset["map_name"]!="Ecopoint: Antarctica") &
    (map_team_heroes_dataset["map_name"]!="Necropolis")
    ]
map_team_heroes_dataset.head()

,map_name,team_name,hero_used
0,Blizzard World,Atlanta Reign,"[(Moira, Baptiste, Mei, Brigitte, Lúcio, Symme..."
1,Blizzard World,Boston Uprising,"[(Baptiste, Mei, Wrecking Ball, Lúcio, McCree,..."
2,Blizzard World,Chengdu Hunters,"[(Mercy, Wrecking Ball, McCree, D.Va, Ana, Win..."
3,Blizzard World,Dallas Fuel,"[(Doomfist, Moira, Baptiste, Mei, Wrecking Bal..."
4,Blizzard World,Florida Mayhem,"[(Baptiste, Brigitte, Mei, Lúcio, McCree, D.Va..."


### Calculate the frequency heroes

In [10]:
def frequency_heroes(map=None, team=None, support=0.6, heroes_num=2):
    if ((map is not None) & (team is not None)):
        try:
            dataset = map_team_heroes_dataset[
                (map_team_heroes_dataset["map_name"]==map) &
                (map_team_heroes_dataset["team_name"]==team)
            ]["hero_used"].agg(sum)
        except:
            error_info = "Wrong map name or team name."
            return error_info
    elif ((map is not None) & (team is None)):
        try:
            dataset = map_team_heroes_dataset.groupby("map_name")["hero_used"].agg(sum)[map]
        except:
            error_info = "Wrong map name."
            return error_info
    elif ((map is None) & (team is not None)):
        try:
            dataset = map_team_heroes_dataset.groupby("team_name")["hero_used"].agg(sum)[team]
        except:
            error_info = "Wrong team name."
            return error_info
    else:
        dataset = map_team_heroes_dataset["hero_used"].agg(sum)
    te = TransactionEncoder()
    te_ary = te.fit(dataset).transform(dataset)
    one_hot_df = pd.DataFrame(te_ary, columns=te.columns_)
    frequent_heroes = apriori(one_hot_df, min_support=support, use_colnames=True)
    frequent_heroes["length"] = frequent_heroes["itemsets"].apply(lambda x: len(x))
    frequent_heroes.sort_values(by=["support"], inplace=True, ascending=False)
    return frequent_heroes[frequent_heroes["length"]==heroes_num]

### Examples

In [6]:
print("Maps:", map_team_heroes_dataset["map_name"].unique())
print()
print("Teams:", map_team_heroes_dataset["team_name"].unique())

Maps: ['Blizzard World' 'Busan' 'Dorado' 'Eichenwalde' 'Hanamura' 'Havana'
 'Hollywood' 'Horizon Lunar Colony' 'Ilios' 'Junkertown' "King's Row"
 'Lijiang Tower' 'Nepal' 'Numbani' 'Oasis' 'Paris' 'Rialto' 'Route 66'
 'Temple of Anubis' 'Volskaya Industries' 'Watchpoint: Gibraltar']

Teams: ['Atlanta Reign' 'Boston Uprising' 'Chengdu Hunters' 'Dallas Fuel'
 'Florida Mayhem' 'Guangzhou Charge' 'Hangzhou Spark' 'Houston Outlaws'
 'London Spitfire' 'Los Angeles Gladiators' 'Los Angeles Valiant'
 'New York Excelsior' 'Paris Eternal' 'Philadelphia Fusion'
 'San Francisco Shock' 'Seoul Dynasty' 'Shanghai Dragons' 'Team D.va'
 'Team Reinhardt' 'Team Universe' 'Toronto Defiant' 'Triple A'
 'Vancouver Titans' 'Washington Justice' 'Brick Movers'
 'Jehong and Students' 'Team Custa' 'Team Jake']


In [11]:
frequency_heroes(map="Blizzard World", team="Chengdu Hunters", support=0.7, heroes_num=2)

,support,itemsets,length
7,0.888889,"(Symmetra, Wrecking Ball)",2
5,0.777778,"(Mercy, Wrecking Ball)",2
6,0.777778,"(Symmetra, Tracer)",2
8,0.777778,"(Tracer, Wrecking Ball)",2
9,0.777778,"(Zenyatta, Wrecking Ball)",2


In [12]:
frequency_heroes(map="Nepal", support=0.5, heroes_num=3)

,support,itemsets,length
17,0.508333,"(Lúcio, Symmetra, Baptiste)",3
18,0.508333,"(Reinhardt, Symmetra, Lúcio)",3


In [13]:
frequency_heroes(team="Paris Eternal", support=0.6, heroes_num=1)

,support,itemsets,length
1,0.734300,(Lúcio),1
0,0.666667,(D.Va),1
2,0.603865,(Tracer),1
3,0.603865,(Wrecking Ball),1


In [22]:
frequency_heroes(support=0.4, heroes_num=2)

,support,itemsets,length
13,0.478739,"(Lúcio, D.Va)",2
15,0.439638,"(D.Va, Wrecking Ball)",2
17,0.436950,"(Tracer, Wrecking Ball)",2
14,0.422287,"(D.Va, Tracer)",2
11,0.417644,"(D.Va, Ana)",2
16,0.408358,"(Lúcio, Wrecking Ball)",2
12,0.407625,"(Tracer, Brigitte)",2
